<a href="https://colab.research.google.com/github/kxk302/Mutation_As_Time_Series/blob/main/Mutation_As_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls '/content/gdrive/MyDrive/Colab Notebooks/Mutation_As_Time_Series_Folder'

In [ ]:
#
# ONLY need to run this ONCE
# Sort the dataset based on Collection_Date and Sample, in ascending order
#

import numpy as np
import pandas as pd

from dateutil import parser

pd.set_option('max_rows', None)

path = '/content/gdrive/MyDrive/Colab Notebooks/Mutation_As_Time_Series_Folder'

df_in = pd.read_csv(path + '/data/data.tsv', sep='\t', names=['Sample', 'Collection_Date', 'UNK1', 'UNK2', 'UNK3', 'POS', 'REF', 'ALT', 'EFFECT', 'CODON', 'TRID', 'AA', 'AF'])

print(type(df_in.Collection_Date[0]))
df_in.Collection_Date = df_in.Collection_Date.apply(lambda x: parser.parse(x))
print(type(df_in.Collection_Date[0]))

df_in.sort_values(by=['Collection_Date', 'Sample'], ascending=[True, True], inplace=True)
print('\n\n')
print(df_in.shape)
print(df_in.head(1000))

# df_in.to_csv(path + '/data/data_sorted.tsv', sep='\t', index=False)

In [ ]:
#
# ONLY need to run this if changing the filtering criteria like start/end dates, or effects
# Filter the dataset, calculate normalized mutation counts, pivot the data, and save to file
#

import numpy as np
import pandas as pd

from dateutil import parser

pd.set_option('max_rows', None)
pd.set_option('max_columns', None)

start_date = '2020-04-13'
end_date = '2021-08-09'
path = '/content/gdrive/MyDrive/Colab Notebooks/Mutation_As_Time_Series_Folder'
file_name = path + '/data/data_sorted.tsv'
separator = '\t'
effects = ['NON_SYNONYMOUS_CODING'] # NON_SYNONYMOUS_CODING -> missense mutation
drop_columns = ['UNK1', 'UNK2', 'UNK3']
atgc_dict = {'A': {'T': 0, 'G': 1, 'C': 2}, 'T': {'G': 3, 'C': 4, 'A': 5}, 'G': {'C': 6, 'A': 7, 'T': 8}, 'C': {'A': 9, 'T':10, 'G': 11}}

# Index of nucleotide changes
#
# 0: A->T
# 1: A->G
# 2: A->C
#
# 3: T->G
# 4: T->C
# 5: T->A
#
# 6: G->C
# 7: G->A
# 8: G->T
#
# 9:  C->A
# 10: C->T
# 11: C->G
#

# Read the input file
df_in = pd.read_csv(file_name, sep=separator)

# Drop the unnecessary columns
df_in.drop(columns=drop_columns, inplace=True)

# Select only rows with mutation type specified in 'effects' list
df_eff = df_in[ df_in.EFFECT.isin(effects) ]

# Select only rows where the Collection_Date falls between start_date and end_date 
df_fil = df_eff[ (df_eff.Collection_Date >= start_date) & (df_eff.Collection_Date <= end_date) ]

print('\n\n')
print('Filtered df shape {}'.format(df_fil.shape))
print('Number of unique dates {}'.format(len(df_fil.Collection_Date.unique())))

print('Calculating nucleotide_change') 
df_fil['nucleotide_change'] = df_fil.apply(lambda x: atgc_dict[x.REF][x.ALT], axis=1)

print('Calculating normalized_nucleotide_change')
normalized_nucleotide_change = df_fil.groupby(df_fil.Collection_Date).nucleotide_change.value_counts() / df_fil.groupby(df_fil.Collection_Date).nucleotide_change.count()

print('\n\n')
print('Type of normalized_nucleotide_change')
print(type(normalized_nucleotide_change))

# Rename to avoid clash when resetting index
print('Name of normalized_nucleotide_change BEFORE rename')
print(normalized_nucleotide_change.name)
normalized_nucleotide_change = normalized_nucleotide_change.rename( 'normalized_nucleotide_change')
print('Name of normalized_nucleotide_change AFTER rename')
print(normalized_nucleotide_change.name)
print('Index of normalized_nucleotide_change')
print(normalized_nucleotide_change.index)

# Convert normalized_nucleotide_change Series to Dataframe
df = normalized_nucleotide_change.to_frame()

print('\n\n')
print('Type of df')
print(type(df))
print('Columns of df')
print(df.columns)
print('df.head(5)')
print(df.head(5))

print('df.index BEFORE reset')
print(df.index)
df.reset_index(inplace=True)
print('df.index AFTER reset')
print(df.index)
print('df.head(5) AFTER reset')
print(df.head(5))

# Pivot the data 
df_piv = pd.pivot_table(df, index='Collection_Date', columns='nucleotide_change', values='normalized_nucleotide_change')
df_piv.fillna(0, inplace=True)

print('\n\n')
print('Pivoted df')
print(df_piv.head(5))
print('Index of df_piv')
print(df_piv.index)
print('Columns of df_piv')
print(df_piv.columns)
print('Shape of df_piv')
print(df_piv.shape)

df_piv.to_csv(path + '/data/data_pivoted_' + start_date + '_' + end_date + '.tsv', sep=separator)

In [ ]:
#
# Create time series prediction dataset from pivoted data
# Depends on how many time steps in the past we look, 
# and how many timestamps in the future we predict
#

import numpy as np
import pandas as pd

from dateutil import parser
from sklearn.model_selection import train_test_split

pd.set_option('max_rows', None)
pd.set_option('max_columns', None)

def create_time_series_dataset(start_date = '2020-04-13', 
                               end_date = '2021-08-09',
                               path = '/content/gdrive/MyDrive/Colab Notebooks/Mutation_As_Time_Series_Folder',
                               data_folder = '/data', 
                               separator = '\t',
                               test_data_percentage = 0.20,
                               num_future = 1,
                               num_past = 14):
  
  file_name = path + data_folder + '/data_pivoted_' + start_date + '_' + end_date + '.tsv'

  # Read the input file
  df_in = pd.read_csv(file_name, sep=separator)

  # Drop the Collection_Date column
  df_in.drop(columns='Collection_Date', inplace=True)

  print(df_in.head(5))
  print(df_in.shape)

  X = []
  Y = []

  num_rows = df_in.shape[0]
  num_cols = df_in.shape[1]

  for i in range(num_past, num_rows - num_future + 1):
    X.append(df_in.iloc[i - num_past:i, 0:num_cols])
    Y.append(df_in.iloc[i + num_future - 1:i + num_future, 0:num_cols])

  X, Y = np.array(X), np.array(Y)

  print('X shape == {}'.format(X.shape))
  print('Y shape == {}'.format(Y.shape))

  X_shape_1 = X.shape[1]
  X_shape_2 = X.shape[2]

  # Must reshape as such, so splitting the data into train/test makes sense
  X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])
  Y = Y.reshape(Y.shape[0], Y.shape[1] * Y.shape[2])
  print('X shape == {}'.format(X.shape))
  print('Y shape == {}'.format(Y.shape))

  trainX, testX, trainY, testY = train_test_split(X, Y, test_size=test_data_percentage)

  # Reshape to the original shape
  trainX = trainX.reshape(trainX.shape[0], X_shape_1, X_shape_2)
  testX = testX.reshape(testX.shape[0], X_shape_1, X_shape_2)

  print('trainX shape == {}'.format(trainX.shape))
  print('trainY shape == {}'.format(trainY.shape))
  print('testX shape == {}'.format(testX.shape))
  print('testY shape == {}'.format(testY.shape))

  return trainX, testX, trainY, testY

In [ ]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error

path = '/content/gdrive/MyDrive/Colab Notebooks/Mutation_As_Time_Series_Folder'
start_date = '2020-04-13'
end_date = '2021-08-09'
data_folder = '/data'
separator = '\t'
test_data_percentage = 0.20
num_future = 1
num_past = 14
epochs=10
batch_size=15
validation_split=0.2

def train_neural_network(path = '/content/gdrive/MyDrive/Colab Notebooks/Mutation_As_Time_Series_Folder',
                         start_date = '2020-04-13',
                         end_date = '2021-08-09',
                         data_folder = '/data',
                         separator = '\t',
                         test_data_percentage = 0.20,
                         num_future = 1,
                         num_past = 14,
                         epochs=10,
                         batch_size=15,
                         validation_split=0.2):

  trainX, testX, trainY, testY = create_time_series_dataset(start_date,
                                                            end_date,
                                                            path,
                                                            data_folder,
                                                            separator,
                                                            test_data_percentage,
                                                            num_future,
                                                            num_past)

  model = Sequential()
  model.add(LSTM(1000, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
  model.add(Dense(750, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(250, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(150, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(50, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(trainY.shape[1]))

  model.compile(optimizer='adam', loss='mse')
  model.summary()

  history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_split=validation_split, verbose=1)
  print('Training loss')
  print(history.history)

  plt.figure(figsize=(15, 6))
  plt.plot(history.history['loss'], lw=3, ls='--', label='Loss')
  plt.plot(history.history['val_loss'], lw=2, ls='-', label='Val Loss')
  plt.xlabel('Epochs', fontsize=15)
  plt.ylabel('Loss', fontsize=15)
  plt.title('MSE, past: ' + str(num_past) + ', future: ' + str(num_future))
  plt.legend()
  plt.savefig(path + '/results/past_' + str(num_past) + '_future_' + str(num_future) + '.png')

  predicted_output = model.predict(testX, verbose=0)
  predicted_mse = mean_squared_error(testY, predicted_output)
  print('MSE of Test dataset')
  print(predicted_mse)

  with open(path + '/results/test_mse_past_' + str(num_past) + '_future_' + str(num_future) + '.txt', 'w') as fp:
    fp.write('MSE of Test dataset, past: ' + str(num_past) + ', future: ' + str(num_future) + '\n')
    fp.write(str(predicted_mse))

In [ ]:
path = '/content/gdrive/MyDrive/Colab Notebooks/Mutation_As_Time_Series_Folder'
start_date = '2020-04-13'
end_date = '2021-08-09'
data_folder = '/data'
separator = '\t'
test_data_percentage = 0.20
epochs=50
batch_size=15
validation_split=0.2

for num_past in [7, 14, 21, 28, 35, 42, 49]:
  for num_future in [1, 3, 6, 9, 12, 15, 18]:

    train_neural_network(path,
                         start_date,
                         end_date,
                         data_folder,
                         separator,
                         test_data_percentage,
                         num_future,
                         num_past,
                         epochs,
                         batch_size,
                         validation_split)

In [ ]:
#
# Graph daily average mutation rate 
#

import pandas as pd

from matplotlib import pyplot as plt
from dateutil import parser

df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Mutation_As_Time_Series_Folder/data/data_pivoted_2020-04-13_2021-08-09.tsv', sep='\t')

x = df.Collection_Date.values.tolist()
x_date = [parser.parse(item) for item in x]

print(len(x))

y1 = df['0']
y2 = df['1']
y3 = df['2']
y4 = df['3']
y5 = df['4']
y6 = df['5']
y7 = df['6']
y8 = df['7']
y9 = df['8']
y10 = df['9']
y11 = df['10']
y12 = df['11']

fig, ax = plt.subplots(6, 2)

fig.set_size_inches(20.0, 20.0)
fig.subplots_adjust(hspace=1.5)

plt.subplot(6, 2, 1)
plt.plot(x, y1, 'r', label='A -> T')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 2)
plt.plot(x, y2, 'b', label='A -> G')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 3)
plt.plot(x, y3, 'g', label='A -> C')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 4)
plt.plot(x, y4, 'r', label='T -> G')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 5)
plt.plot(x, y5, 'b', label='T -> C')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 6)
plt.plot(x, y6, 'g', label='T -> A')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 7)
plt.plot(x, y7, 'r', label='G -> C')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 8)
plt.plot(x, y8, 'b', label='G -> A')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 9)
plt.plot(x, y9, 'g', label='G -> T')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 10)
plt.plot(x, y10, 'r', label='C -> A')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 11)
plt.plot(x, y11, 'b', label='C -> T')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.subplot(6, 2, 12)
plt.plot(x, y12, 'g', label='C -> G')
plt.legend()
plt.xticks(x[::20], rotation=25)

plt.show()

In [ ]:
'''
# Make a single prediction
testX_0 = testX[0].reshape(1, testX.shape[1], testX.shape[2])
testY_0 = testY[0].reshape(1, testY.shape[1])

print('Predicted output')
predicted_output_0 = model.predict(testX_0, verbose=0)
print(predicted_output_0)
print('Actual output')
print(testY_0)
print('MSE of prediction')
np.sqrt(mean_squared_error(testY_0, predicted_output_0))
'''